In [ ]:
import os, sys
import numpy as np
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, Adam
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
torch.manual_seed(12345)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

## Preliminary

In [ ]:
na_list = ['A', 'C', 'G', 'T'] #nucleic acids
aa_list = ['R', 'L', 'S', 'A', 'G', 'P', 'T', 'V', 'N', 'D', 'C', 'Q', 'E', 'H', 'I', 'K', 'M', 'F', 'W', 'Y'] #amino acids
NNK_freq = [0.09375]*3 + [0.0625]*5 + [0.03125]*13 #freq of 21 NNK codons including the stop codon
sum_20 = 0.0625*5 + 0.09375*3 + 0.03125*12 #sum of freq without the stop codon
pvals = [0.09375/sum_20]*3 + [0.0625/sum_20]*5 + [0.03125/sum_20]*12 #normalize freq for 20 codons
pvals = [0.09375/sum_20]*3 + [0.0625/sum_20]*5 + [0.03125/sum_20]*11 + \
        [1- sum([0.09375/sum_20]*3 + [0.0625/sum_20]*5 + [0.03125/sum_20]*11)] 
        #adjust sum to 1 due to numerical issue
aa_dict = dict(zip(aa_list, pvals))

## Dataset

In [ ]:
aptamer_dataset_file = "../data/aptamer_dataset.json"

def construct_dataset():
    with open(aptamer_dataset_file, 'r') as f:
        aptamer_data = json.load(f)
    full_dataset = []
    aptamers = []
    peptides = []
    for aptamer in aptamer_data:
        peptides = aptamer_data[aptamer]
        if aptamer == "CTTTGTAATTGGTTCTGAGTTCCGTTGTGGGAGGAACATG": #took out aptamer control
            continue
        for peptide, _ in peptides:
            peptide = peptide.replace("_", "") #removed stop codons
            if "RRRRRR" in peptide: #took out peptide control
                continue
            if len(aptamer) == 40 and len(peptide) == 8: #making sure right length
                full_dataset.append((aptamer, peptide))
    full_dataset = list(set(full_dataset)) #removed duplicates
    for pair in full_dataset:
        aptamers.append(pair[0])
        peptides.append(pair[1])
    return full_dataset, aptamers, peptides 

In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, training_set):
        super(TrainDataset, self).__init__() 
        self.training_set = training_set
        
    def __len__(self):
        return len(self.training_set)

    def __getitem__(self, idx):
        aptamer, peptide = self.training_set[idx]
        return aptamer, peptide
    
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, test_set):
        super(TestDataset, self).__init__() 
        self.test_set = test_set
        
    def __len__(self):
        return len(self.test_set)

    def __getitem__(self, idx):
        aptamer, peptide = self.test_set[idx]
        return aptamer, peptide

In [ ]:
full_dataset, aptamers, peptides = construct_dataset()
n = len(full_dataset)
training_set = full_dataset[:int(0.8*n)]
test_set = full_dataset[int(0.8*n):]
train_dataset = TrainDataset(training_set)
test_dataset = TestDataset(test_set)
train_loader = torch.utils.data.DataLoader(train_dataset)
test_loader = torch.utils.data.DataLoader(test_dataset)

## One-hot encoding

In [ ]:
## Takes a peptide and aptamer sequence and converts to one-hot matrix
def one_hot(sequence, seq_type='peptide'):
    if seq_type == 'peptide':
        letters = aa_list
    else:
        letters = na_list
    one_hot = np.zeros((len(sequence), len(letters)))
    for i in range(len(sequence)):
        char = sequence[i]
        for _ in range(len(letters)):
            idx = letters.index(char)
            one_hot[i][idx] = 1
    return one_hot

## NN Models

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cnn_apt_1 = nn.Conv2d(40, 20, 1)
        self.cnn_apt_2 = nn.Conv2d(20, 10, 1)
        self.cnn_apt_3 = nn.Conv2d(10, 1, 1)
        self.fc_apt_1 = nn.Linear(160, 1)
        
        self.cnn_pep_1 = nn.Conv2d(8, 4, 1)
        self.cnn_pep_2 = nn.Conv2d(4, 3, 1)
        self.fc_pep_1 = nn.Linear(64, 1)
        
        self.pool = nn.MaxPool2d(1, 1)
        self.relu = nn.ReLU()
                
        self.sequential_pep = nn.Sequential(self.cnn_pep_1,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_pep_2)
        
        self.sequential_apt = nn.Sequential(self.cnn_apt_1, 
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_apt_2, 
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_apt_3)
        
        self.fc1 = nn.Linear(64, 1)
        
    def forward(self, apt, pep):
        apt = self.sequential_apt(apt).cuda()
        pep = self.sequential_pep(pep).cuda()
        
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        
        x = torch.cat((apt, pep), 1)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.cnn_apt_1 = nn.Conv2d(40, 100, (3,1))
        self.cnn_pep_1 = nn.Conv2d(8, 40, (2,1))
        self.pool = nn.MaxPool2d(1, 1)
        self.relu = nn.ReLU()
                
        self.sequential_pep = nn.Sequential(self.cnn_pep_1,
                                            self.relu, 
                                            self.pool)
        
        self.sequential_apt = nn.Sequential(self.cnn_apt_1, 
                                            self.relu, 
                                            self.pool)
        
        self.fc1 = nn.Linear(960, 1)
        
    def forward(self, apt, pep):
        apt = self.sequential_apt(apt)
        pep = self.sequential_pep(pep)
        
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        
        x = torch.cat((apt, pep), 1)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
        nn.init.zeros_(m.bias.data)

## Sampling

In [ ]:
# Sample x from P_X (assume peptides follow NNK)
def get_x():
    x_idx = np.random.choice(20, 7, p=pvals)
    x = "M"
    for i in x_idx:
        x += aa_list[i]
    return x

# Sample y from P_Y (assume apatamers follow uniform)
def get_y():
    y_idx = np.random.randint(0, 4, 40)
    y = ""
    for i in y_idx:
        y += na_list[i]
    return y

# Generate uniformly from S without replacement
def get_xy(k):
    samples = [full_dataset[i] for i in np.random.choice(len(full_dataset), k, replace=False)]
    return samples

# S' contains S with double the size of S (domain for Importance Sampling)
def get_S_prime(k):
    S_prime = full_dataset[:]
    for _ in range(k):
        S_prime.append((get_y(), get_x()))
    return list(set(S_prime))

# Sample from S' without replacement
def get_xy_prime(k):
    samples = [S_prime[i] for i in np.random.choice(len(S_prime), k, replace=False)]
    return samples

# Returns pmf of a peptide
def get_x_pmf(x):
    pmf = 1
    for char in x[1:]: #skips first char "M"
        pmf *= aa_dict[char]
    return pmf

# Returns pmf of an aptamer
def get_y_pmf():
    return 0.25**40

S_prime = get_S_prime(n)

## SGD

In [ ]:
# Convert a pair to one-hot tensor
def convert(apt, pep): 
    apt = one_hot(apt, seq_type='aptamer') #(40, 4)
    pep = one_hot(pep, seq_type='peptide') #(8, 20)
    apt = torch.FloatTensor(np.reshape(apt, (1, apt.shape[0], apt.shape[1], 1))).cuda() #(1, 40, 4, 1)
    pep = torch.FloatTensor(np.reshape(pep, (1, pep.shape[0], pep.shape[1], 1))).cuda() #(1, 8, 20, 1)
    return apt, pep

def update(type="original"):
    if type == "original":
        xy = get_xy(1)[0]
    else:
        xy = get_xy_prime(1)[0]
    x, y = convert(xy[0], xy[1])
    x.requires_grad=True
    y.requires_grad=True
    x = x.cuda()
    y = y.cuda()
    out = model(x, y)
    return xy, out

def apply_param_grad(grads1, grads2, fn):
    gs = []
    for grad1, grad2 in zip(grads1, grads2):
        gs.append(fn(grad1, grad2))
    return gs

def sgd(t=1000, #num of iter
        lamb=1e-5, #hyperparam
        gamma=1e-3): #step size
    optim = SGD(model.parameters(), lr=gamma)
    model.train()
    for a, _ in enumerate(tqdm.tqdm(range(t))):
        optim.zero_grad()
        xy, out = update()
        log_out = torch.log(out)
        log_out.backward(retain_graph=True)
        g1 = []
        for param in model.parameters():
            g1.append(param.grad)
        
        optim.zero_grad()
        xy_prime, out_prime = update("prime")
        out_prime = out_prime * get_x_pmf(xy_prime[0]) * get_y_pmf() * 2 * n
        out_prime.backward()
        g2 = []
        for param in model.parameters():
            g2.append(param.grad)
        
        const = 0 if xy_prime in full_dataset else 1 #indicator
        gs = apply_param_grad(g1, g2, lambda g1, g2: lamb*const*g2 - g1)
        
        #Update the weights according to SGD
        for param, g in zip(model.parameters(), gs):
            param.grad = g
        optim.step()

In [ ]:
# Recall on train set of size k to test for overfitting
def recall_train(k):
    model.eval()
    samples = get_xy(k)
    correct = 0
    train_recall_outputs = []
    for (apt, pep) in samples:
        apt, pep = convert(apt, pep)
        out = model(apt, pep).cpu().detach().numpy().flatten()[0]
        train_recall_outputs.append(out)
        if out > 0.5:
            correct += 1
    train_recall = 100*correct/k #recall rate of k samples from training set
    return train_recall, train_recall_outputs #list of k outputs


# Recall on test set of size k
def recall_test(k):
    model.eval()
    correct = 0
    count = 0
    test_recall_outputs = []
    for _, (aptamer, peptide) in enumerate(tqdm.tqdm(test_loader)):
        if count > k:
            break
        apt, pep = convert(aptamer[0], peptide[0])
        output = model(apt, pep).cpu().detach().numpy().flatten()[0]
        test_recall_outputs.append(output)
        if output > 0.5:
            correct += 1
        count += 1
    recall_test = 100*correct/k #recall rate of k samples from test set
    return recall_test, test_recall_outputs #list of k outputs


# Eval on m new unseen pairs(not in our dataset)
def eval_unknown(m):
    model.eval()
    eval_unknown_outputs = []
    count = 0
    for _ in range(m):
        x, y = get_x(), get_y()
        apt, pep = convert(y, x)
        output = model(apt, pep).cpu().detach().numpy().flatten()[0]
        eval_unknown_outputs.append(output)
        if output < 0.5:
            count += 1
    precision = 100 * count / m
    return precision, eval_unknown_outputs #list of m outputs

In [ ]:
gammas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
lambdas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
train_recalls = []
test_recalls = []
train_scores = []
test_scores = []
train_cdfs = []
test_cdfs = []

m = int(1e6) # number of unknown samples
k = m//10 # number of binding samples (test set size is 118262, k is just some limit we set)

for g in range(len(gammas)):
    for l in range(len(lambdas)):
        model = SimpleConvNet()
        model.apply(weights_init)
        model.cuda()
        
        print("=============Training=======================")
        sgd(t=1000, gamma=gammas[g], lamb=lambdas[l])
        
        print("=============Evaluating train===============")
        train_recall, train_recall_outputs = recall_train(k)
        print("Gamma: ", "%.5f" % gammas[g], "Lambda: ", "%.5f" % lambdas[l], \
              "Train recall: ", "%.2f" % train_recall)
        
        print("=============Evaluating test================")
        test_recall, test_recall_outputs = recall_test(k)
        print("Gamma: ", "%.5f" % gammas[g], "Lambda: ", "%.5f" % lambdas[l], \
              "Test recall: ", "%.2f" % test_recall)
        
        print("=============Evaluating unknown=============")
        precision, eval_unknown_outputs = eval_unknown(m)
        
        train_score = np.asarray(eval_unknown_outputs + train_recall_outputs)
        train_scores.append(train_score)
        
        test_score = np.asarray(eval_unknown_outputs + test_recall_outputs)
        test_scores.append(test_score)
        
        train_cdf = np.sum(np.cumsum(train_score), dtype=float)/(np.sum(train_score)*len(train_score))
        test_cdf = np.sum(np.cumsum(test_score), dtype=float)/(np.sum(test_score)*len(test_score))
        print("G: ", "%.5f" % gammas[g], "L: ", "%.5f" % lambdas[l], \
              "Train CDF: ", "%.3f" % train_cdf, "Test CDF: ", "%.3f" % test_cdf)
        
        train_recalls.append(train_recall)
        test_recalls.append(test_recall)
        train_cdfs.append((gammas[g], lambdas[l], train_cdf))
        test_cdfs.append((gammas[g], lambdas[l], test_cdf))

print("Train CDFs: ", train_cdfs)
print("Test CDFs: ", test_cdfs)
    

In [ ]:
# AUC Plot(one config for now)
def cdf(scores, i):
    #tot = len(gammas) * len(lambdas)
    #for i in range(tot):
    #unknown, binding = scores[i]
    #total = unknown + binding
    plt.hist(scores, 100, histtype='step', density=True, cumulative=True)
    g = gammas[i//len(lambdas)]
    l = lambdas[i%len(lambdas)]
    label = 'lambda =%.5f' % l  + ' gamma =%.5f' % g
    plt.legend([label])
    plt.show()

i = 0
cdf(test_scores[i], i)

In [ ]:
### Write to file when output limit exceeded
# with open("./scores.txt", "a") as f:
#     f.write(str(scores[-1]))
#     f.close()

### Table of recalls with different params
# idx = sorted(range(len(recalls)), key=lambda k: recalls[k])
# for i in idx:
#     g = gammas[i//len(lambdas)]
#     l = lambdas[i%len(lambdas)]
#     print("Gamma: ", "%.5f" % g, "Lambda: ", "%.5f" % l, \
#           "Recall: ", "%.2f" % recalls[i], "Precision: ", "%.2f" % precisions[i])

### Heatmap of recalls
# mat = sns.heatmap(M, vmin=0, vmax=100)
# plt.show()

### Test other NN models
# model = SimpleConvNet()
# model.apply(weights_init)
# model.cuda()
# print("Training...")
# sgd(t=1000, gamma=0.01, lamb=0.01)
# print("Evaluating...")
# # use for AUC
# recall, binding_outputs = recall_eval(50)
# unknown_outputs = evaluate(500)
# # use for heatmap
# print("Recall with gamma: "+ str(0.01) + " , lambda: " + str(0.01) + " recall: ", '%.2f'% recall)